In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
import collections
from tqdm import tqdm
from datetime import datetime
import random
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Himanshu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
data_location = "E:/datasets/word2vec/text8/text8"
window_size = 4
tablesize = 1000000

In [3]:
data = open(data_location, "rb").read()
data = data.decode("utf8")

In [4]:
token_index = collections.OrderedDict()
token_counts = collections.OrderedDict()

j=1
tokens = data.split(" ")[: 50000]
tokens = [token for token in tokens if token not in stopwords]
for index, token in enumerate(tokens):
    if token in token_index:
        token_counts[token] += 1
    else:
        token_index[token] = j
        token_counts[token] = 1
        j += 1
token_index["UNK"] = 0

rev_token_index = {j: i for i, j in token_index.items()}
sampling_freq = np.zeros(len(token_index))
for i in range(len(token_index)):
    word = rev_token_index[i]
    if word in token_counts:
        freq = token_counts[word] ** 0.75
    else:
        freq = 0.0
    sampling_freq[i] = freq

sampling_freq /= np.sum(sampling_freq)
sampling_freq = np.cumsum(sampling_freq) * tablesize
j = 0
sampling_table = []
for i in range(tablesize):
    while i > sampling_freq[j]:
        j = j + 1
    sampling_table.append(j)

In [5]:
inputs = []
outputs = []
for i in range(len(tokens)):
    inputs.append(token_index[tokens[i]])
    window_start_index = i - window_size
    window_start_index = 0 if window_start_index < 0 else window_start_index
    window_end_index = i + window_size
    window_end_index = len(tokens) if window_end_index >= len(tokens) else window_end_index
    output = []
    for j in range(window_start_index, window_end_index):
        if j != i:
            output.append(token_index[tokens[j]])
    outputs.append(output)
outputs = np.array(outputs)

In [6]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class Skipgram:
    def __init__(self, num_features, vocab_size, negative_sample_table):
        self.num_features = num_features
        self.w1 = np.random.uniform(0.0, 0.5, size=(vocab_size, self.num_features))
        self.w2 = np.random.uniform(0.0, 0.5, size=(vocab_size, self.num_features))
        self.vocab_size = vocab_size
        self.negative_sample_table = negative_sample_table
    
    def _forward_pass(self, model_input):
        self.h1 = np.dot(model_input, self.w1)
        self.h2 = np.dot(self.h1, self.w2.transpose())
    
    def _backward_pass(self, model_input, model_output, lr, num_negative_samples):
        negative_samples = self.get_negative_sample(num_negative_samples, model_output)
        sig_h2o = sigmoid(self.h2[model_output])
        sig_h2k = sigmoid(-1 * self.h2[negative_samples])
        delta_j_h2o = sig_h2o - 1
        delta_j_h2k = 1 - sig_h2k
        self.w1 -= np.dot(model_input, (sum(delta_j_h2o) + sum(delta_j_h2k)) * self.w2) * lr
        self.w2[model_output] -= delta_j_h2o.reshape(len(delta_j_h2o), 1) * self.h1 * lr
        self.w2[negative_samples] -= delta_j_h2k.reshape(len(delta_j_h2k), 1) * self.h1 * lr
        return sig_h2o, sig_h2k
    
    def get_negative_sample(self, num_samples, model_output):
        negative_samples = []
        while len(negative_samples) < num_samples:
            negative_sample = self.negative_sample_table[random.randint(0, tablesize - 1)]
            if negative_sample not in model_output:
                negative_samples.append(negative_sample)
        return negative_samples
    
    def fit(self, model_inputs, model_outputs, lr, epochs, num_negative_samples):
        for epoch in range(epochs):
            loss = 0.0
            sample_indices = np.arange(len(model_inputs) - 1)
            np.random.shuffle(sample_indices)
            for i in tqdm(sample_indices):
                model_input = np.zeros((self.vocab_size))
                model_input[model_inputs[i]] = 1.0
                self._forward_pass(model_input=model_input)
                sig_h2o, sig_h2k = self._backward_pass(model_input, model_outputs[i], lr, num_negative_samples)
                loss += -1 * sum(np.log(sig_h2o))
                loss += -1 * sum(np.log(sig_h2k))
            print("epochs: {0} loss: {1}".format(str(epoch+1), str(loss)))
            pickle.dump(self, open("skipgram_ns_epoch_{0}".format(str(epoch + 1)), "wb"))
    
    def most_similar(self, input_word, token_index):
        assert input_word in token_index
        word_vectors = (self.w1 + self.w2) / 2
        prediction = cosine_similarity(word_vectors[token_index[input_word]].reshape(1, -1), word_vectors)[0]
        sorted_index = np.argsort(prediction)[::-1].tolist()
        rev_token_index = {j: i for i, j in token_index.items()}
        for index in range(10):
            print(rev_token_index[sorted_index[index]])

In [7]:
model = Skipgram(100, len(token_index) + 1, sampling_table)
model.fit(inputs, outputs, 0.001, 300, 200)

  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

epochs: 1 loss: 958902.0838597687


  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

epochs: 2 loss: 946957.8322672453


  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

...

epochs: 115 loss: 1047014.5237040354


  0%|                                                                                                                                      | 0/30576 [00:00<?, ?it/s]

epochs: 116 loss: 1050501.7644473643


 59%|███████████████████████████████████████████████████████████████████████▊                                                  | 17988/30576 [03:36<02:57, 71.10it/s]

KeyboardInterrupt: 

In [ ]:
model.most_similar("coup", token_index)

In [ ]:
token_index